In [1]:
import pickle
import json
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.base import BaseEstimator, TransformerMixin
import category_encoders
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer, FunctionTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from utils import DataCleaner

/home/alex/miniconda3/envs/shell/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/alex/miniconda3/envs/shell/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
pipeline = joblib.load('pipeline.pickle')
with open('dtypes.pickle', 'rb') as fh:
    dtypes = pickle.load(fh)

with open('columns.json', 'r') as fh:
    columns = json.load(fh)

# Build pipeline

In [3]:
!pwd

/mnt/c/Users/Alexandre/Google Drive/Documents/LDSSA/heroku-model-deploy


In [4]:
df_x = pd.read_csv('../batch2-capstone/data/X_train.csv')
df_x["age_in_years"] = df_x["age_in_years"].astype(np.float)

y = np.genfromtxt('../batch2-capstone/data/y_train.csv')

In [5]:
cleaner = DataCleaner(50).fit(df_x,y)

In [6]:
asd=pd.DataFrame.from_dict(
{'m_or_f': [np.nan],
 'person_attributes': [np.nan],
 'seat': ['front_left'],
 'other_person_location': [np.nan],
 'other_factor_1': [np.nan],
 'other_factor_2': [np.nan],
 'other_factor_3': [np.nan],
 'age_in_years': [np.nan]}).astype(df_x.dtypes)

## Overall pipeline

In [7]:
best_forest_parameters = {'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 9,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


best_forest = RandomForestClassifier(**best_forest_parameters)

In [8]:
pipeline = make_pipeline(
    DataCleaner(30),
    Imputer(strategy='mean'),
    RandomForestClassifier(**best_forest_parameters)
)
pipeline.fit(df_x, y)

Pipeline(memory=None,
     steps=[('datacleaner', DataCleaner(n_min=30)), ('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_node...estimators=9, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [9]:
%timeit pipeline.predict_proba(asd)

161 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
pipeline.predict_proba(asd)[0,1]

0.36411787800816947

## Serialize!!

In [11]:
#column names
with open('columns.json', 'w') as fh:
    json.dump(df_x.columns.tolist(), fh)

In [12]:
#column types (don't forget to convert first age to np.float because int can't be nan)
with open('dtypes.pickle', 'wb') as fh:
    pickle.dump(df_x.dtypes, fh)

In [13]:
from sklearn.externals import joblib
joblib.dump(pipeline, 'pipeline.pickle') 

['pipeline.pickle']

In [14]:
__name__

'__main__'

In [15]:
!ls -la

total 4484
drwxrwxrwx 1 alex alex     512 Sep 16 20:27 .
drwxrwxrwx 1 alex alex     512 Sep 16 20:19 ..
-rwxrwxrwx 1 alex alex    6510 Sep 16 20:26 app.py
-rwxrwxrwx 1 alex alex     134 Sep 16 20:31 columns.json
-rwxrwxrwx 1 alex alex    6665 Sep 16 13:12 Deserialize and use.ipynb
-rwxrwxrwx 1 alex alex     878 Sep 16 20:31 dtypes.pickle
drwxrwxrwx 1 alex alex     512 Sep 16 20:27 .git
-rwxrwxrwx 1 alex alex    1285 Sep  9 03:04 .gitignore
drwxrwxrwx 1 alex alex     512 Sep 16 20:27 .idea
drwxrwxrwx 1 alex alex     512 Sep 16 16:28 .ipynb_checkpoints
-rwxrwxrwx 1 alex alex    1114 Sep  9 03:04 LICENSE
drwxrwxrwx 1 alex alex     512 Sep  9 05:31 model
-rwxrwxrwx 1 alex alex 4390159 Sep 16 20:31 pipeline.pickle
-rwxrwxrwx 1 alex alex   12288 Sep 16 20:27 predictions.db
-rwxrwxrwx 1 alex alex      25 Sep  9 03:04 Procfile
drwxrwxrwx 1 alex alex     512 Sep 16 20:31 __pycache__
-rwxrwxrwx 1 alex alex   29835 Sep  9 03:04 README.md
-rwxrwxrwx 1 alex alex      90 Sep  9 03:04 requirements.tx